In [ ]:
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

device = torch.device('cpu')

batch_size = 1

trans = transforms.Compose([transforms.Resize((256, 256)), 
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.ImageFolder(root = "../input_data", 
					transform = trans
                    )

dataLoader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)


In [2]:
from models.e4e.encoders.psp_encoders import Encoder4Editing
from configs import paths_config
from argparse import Namespace
import torch

ckpt = torch.load(paths_config.e4e, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = paths_config.e4e
opts = Namespace(**opts)

encoder = Encoder4Editing(num_layers=50, mode='ir_se', opts=opts).to(device)



In [3]:
from sklearn.linear_model import SGDClassifier
import numpy as np

model = SGDClassifier(
    loss="log_loss",  # Logistic Regression을 위한 손실 함수
    penalty="elasticnet",
    l1_ratio=0.1,
    max_iter=1,  # 각 배치에서 1번의 에포크
    warm_start=True  # 이전 상태를 유지하며 학습
)

In [ ]:
from tqdm import tqdm

classes = np.unique([0, 1])

for epoch in range(5):
    for src, tgt in tqdm(dataLoader):
        
        out = encoder(src).reshape(batch_size, -1).detach().numpy()
        tgt = tgt.detach().numpy()

        model.partial_fit(out, tgt, classes=classes)
            

direction = model.coef_.reshape(18, 512)
np.save(f'w_direction.npy', direction)

100%|██████████| 75720/75720 [11:32:37<00:00,  1.82it/s]  
